In [1]:
import pandas as pd
import numpy as np
#from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, StackingClassifier
#from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
#from bayes_opt import BayesianOptimization

#from sklearn.model_selection import StratifiedKFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import category_encoders as ce

#from imblearn.combine import SMOTETomek

In [2]:
train_values = pd.read_csv('train_values.csv')
train_labels = pd.read_csv('train_labels.csv')

train_values = train_values.applymap(lambda col: np.nan if col == 995 else col)

drop_vars = ['building_id',
             'has_superstructure_bamboo',                
             'has_superstructure_mud_mortar_brick',       
             'has_superstructure_adobe_mud',              
             'legal_ownership_status',                    
             'has_secondary_use',                         
             'has_superstructure_stone_flag',             
             'has_superstructure_rc_non_engineered',      
             'has_superstructure_rc_engineered',          
             'has_secondary_use_agriculture',             
             'has_superstructure_cement_mortar_stone',    
             'has_secondary_use_hotel',                   
             'has_superstructure_other',                
             'has_secondary_use_rental',                  
             'has_secondary_use_other',                   
             'has_secondary_use_industry',               
             'has_secondary_use_institution',             
             'has_secondary_use_school',                  
             'has_secondary_use_health_post',            
             'has_secondary_use_gov_office',          
             'has_secondary_use_use_police' ]


train_values = train_values.drop(drop_vars, axis = 1)

targ_enc = ce.TargetEncoder()
targ_enc.fit(train_values, train_labels['damage_grade'])
train_values = targ_enc.transform(train_values)

imp = IterativeImputer(max_iter=20, min_value = 0)
imp.fit(train_values)

train_values = pd.DataFrame(imp.transform(train_values), columns = train_values.columns)

/home/matt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [3]:
dtrain_full = xgb.DMatrix(train_values, train_labels['damage_grade'].astype(int))

In [4]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 23456)

In [ ]:
xgb_params = {'objective': 'multi:softmax',
              'gamma': 0.001369048068388758,
              'eta': 0.11,
              'max_depth': 20,
              'min_child_weight': 9,
              'num_class' : 4,
              'subsample' : .85,
              'colsample_bytree' : .9,
              'seed':23456
 }


In [ ]:
f1_scores = []
boost_rounds = []

for train_idx, test_idx in skf.split(train_values, train_labels.damage_grade):
    
    #smote = SMOTETomek(sampling_strategy = {1 : 30000}, random_state = 23456)
    #X, y = smote.fit_resample(train_values.iloc[train_idx], train_labels.iloc[train_idx, 1])
    
    #dtrain = xgb.DMatrix(X, y)
    dtrain = dtrain_full.slice(train_idx)
    dtest  = dtrain_full.slice(test_idx)
    
    model = xgb.train(xgb_params,
                      dtrain,
                      num_boost_round = 999,
                      evals = [(dtest, 'test_set')],
                      early_stopping_rounds = 10,
                      verbose_eval = False
                  )
    boost_rounds.append(model.best_iteration + 1)
    print("Boost Rounds: {}".format(model.best_iteration + 1))
    boost_round = model.best_iteration + 1
    
    best_model = xgb.train(xgb_params,
                      dtrain,
                      num_boost_round = boost_round,
                      #evals = [(dtest, 'test_set')],
                      #early_stopping_rounds = 10,
                      verbose_eval = False
                  ) 
    preds = best_model.predict(dtest)
    
    print('F1: {}'.format(f1_score(dtest.get_label(), preds, average = 'micro')))
    f1_scores.append(f1_score(dtest.get_label(), preds, average = 'micro'))

print(f1_scores)
print("Mean F1: {}".format(np.mean(f1_scores)))
print("Mean Boost Rounds: {}".format(np.mean(boost_rounds)))
# score to beat
#Mean F1: 0.743968746612887

Mean F1: 0.7451467927006086
Mean Boost Rounds: 113.8

In [15]:
test = pd.read_csv('test_values.csv')
mtest = test.drop(drop_vars, axis = 1)
mtest = targ_enc.transform(mtest)
mtest = pd.DataFrame(imp.transform(mtest), columns = mtest.columns)
dtest = xgb.DMatrix(mtest)

votes_df = pd.DataFrame(test.iloc[:,0])

In [6]:
seeds = [23456, 45678, 56789, 12345, 98765, 87654, 76543, 54321, 13579, 246801, 57911, 791113]

In [18]:
for idx, seed in enumerate(seeds):
    print("Training model {}".format(idx + 1))
    
    xgb_params = {'objective': 'multi:softmax',
                  'gamma': 0.001369048068388758,
                  'eta': 0.11,
                  'max_depth': 20,
                  'min_child_weight': 9,
                  'num_class' : 4,
                  'subsample' : .85,
                  'colsample_bytree' : .9,
                  'seed': seed
                 }
    
    boost_rounds = []
    
    for train_idx, test_idx in skf.split(train_values, train_labels.damage_grade):
            dtrain = dtrain_full.slice(train_idx)
            dtest_part  = dtrain_full.slice(test_idx)
            
            model = xgb.train(xgb_params,
                              dtrain,
                              num_boost_round = 999,
                              evals = [(dtest_part, 'test_set')],
                              early_stopping_rounds = 10,
                              verbose_eval = False
                          )
            
            boost_rounds.append(model.best_iteration + 1)
    
    best_model = xgb.train(xgb_params,
                           dtrain_full,
                           num_boost_round = np.round(np.mean(boost_rounds)).astype(int),
                           verbose_eval = False
                           ) 
    
    col_nam = ["model", str(idx + 1)]
    col_nam = "_".join(col_nam)
    
    votes_df[col_nam] = best_model.predict(dtest)


Training model 1
Training model 2
Training model 3
Training model 4
Training model 5
Training model 6
Training model 7
Training model 8
Training model 9
Training model 10
Training model 11
Training model 12


In [29]:
votes_df['damage_grade'] = votes_df.mode(axis = 1).iloc[:, 0].values

In [31]:
votes_df["damage_grade"] = votes_df["damage_grade"].astype(int)

In [32]:
votes_df[['building_id', 'damage_grade']].to_csv("voting_xgboost_model.csv", index = False)

In [ ]:
best_model = xgb.train(xgb_params, dtrain_full, num_boost_round = np.round(np.mean(boost_rounds)).astype(int))

In [ ]:


test["damage_grade"] = best_model.predict(dtest)
test["damage_grade"] = test["damage_grade"].astype(int)
print(test.value_counts('damage_grade')) # check that preds look ok

In [ ]:
test[['building_id', 'damage_grade']].to_csv("xgb_model_max_depth_20.csv", index = False)

In [33]:
175/3539

0.04944899689177734